In [25]:
%pip install holoviews

In [29]:
%pip install networkx

In [30]:
import re
import requests
import pandas as pd
import holoviews as hv
import networkx as nx
from holoviews import opts
from bs4 import BeautifulSoup
from urllib.request import urljoin


In [34]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

sites = ["https://www.google.com/"]  # Change sites to a list
depth = 3
max_child = 10

def crawl(sites, depth=1, current_depth=0, max_child=10):
    def get_links(site):
        response = requests.get(site)
        soup = BeautifulSoup(response.text, 'html.parser')
        tags = soup.find_all('a')
        links = [urljoin(site, tag.get('href')) for tag in tags]
        return pd.Series(links).drop_duplicates().head(max_child)

    data = [{'link': site, 'parent': site, 'depth': 0} for site in sites]
    df = pd.DataFrame(data)

    for i in range(depth):
        current_sites = df[df['depth'] == i]['link']
        new_sites = []
        for site in current_sites:
            links = get_links(site)
            new_sites.extend([{'link': link, 'parent': site, 'depth': i + 1} for link in links])
        df = df.append(new_sites, ignore_index=True)
        df = df.drop_duplicates('link')

    return df

result = crawl(sites, depth=depth, max_child=max_child)

AttributeError: 'DataFrame' object has no attribute 'append'

In [35]:
sites = "https://www.google.com/" #@param {type:"string"}
depth = 3 #@param {type:"slider", min:1, max:10, step:1}
max_child = 10 #@param {type:"slider", min:1, max:100, step:1}

def crawl(sites, depth=1, current_depth=0, max_child=10):
    get_links = lambda site: pd.Series(urljoin(site, tag.get('href')) for tag in BeautifulSoup(requests.get(site).text).find_all('a')).drop_duplicates().head(max_child)
    
    sites = pd.concat([sites,pd.DataFrame([{'link': link, 'parent': site, 'depth': current_depth + 1}
                                       for site in sites[sites.depth == current_depth].link
                                       for link in get_links(site)])], 
                         ignore_index=True).drop_duplicates('link')
    
    if depth > current_depth + 1:
        return crawl(sites, depth=depth, current_depth=current_depth+1, max_child=max_child)
    else:
        return sites


result = crawl(sites=pd.DataFrame([{'link': site, 'depth': 0, 'parent': site} for site in sites.split()]), 
               max_child=max_child,
               depth=depth)

In [36]:
result

,link,depth,parent
0,https://www.google.com/,0,https://www.google.com/
1,https://www.google.com/imghp?hl=km&tab=wi,1,https://www.google.com/
2,https://maps.google.com.kh/maps?hl=km&tab=wl,1,https://www.google.com/
3,https://play.google.com/?hl=km&tab=w8,1,https://www.google.com/
4,https://news.google.com/?tab=wn,1,https://www.google.com/
...,...,...,...
540,https://accounts.google.com/ServiceLogin?hl=en...,3,https://support.google.com/websearch/?p=displa...
543,https://support.google.com/websearch/community...,3,https://support.google.com/websearch/?p=displa...
546,https://support.google.com/websearch/?p=result...,3,https://support.google.com/websearch/?p=result...
550,https://accounts.google.com/ServiceLogin?hl=en...,3,https://support.google.com/websearch/?p=result...


In [ ]:
# this is just the example scenarios
# from the results you can see the links "https://support.google.com/websearch", "https://play.google.com"
# it could be the all services google site

In [44]:
#Task : write the regular expression to get all the services of google
import re

texts = result['link']
google_ser = r'https://[^\s]+google\.com'
google_services = [] 
for text in texts:
    match = re.findall(google_ser, str(text))
    google_services.extend(match)
for service in google_services:
    print(service)

https://www.google.com
https://www.google.com
https://maps.google.com
https://play.google.com
https://news.google.com
https://mail.google.com
https://drive.google.com
https://calendar.google.com
https://www.google.com
https://www.google.com
https://www.google.com
https://maps.google.com
https://play.google.com
https://news.google.com
https://mail.google.com
https://drive.google.com
https://calendar.google.com
https://www.google.com
https://www.google.com
https://support.google.com
https://play.google.com
https://policies.google.com
https://myaccount.google.com
https://play.google.com
https://play.google.com
https://play.google.com
https://play.google.com
https://play.google.com
https://play.google.com
https://play.google.com
https://news.google.com
https://www.google.com
https://accounts.google.com/ServiceLogin?passive=1209600&continue=https://news.google.com/home?tab%3Dwn%26hl%3Den-US%26gl%3DUS%26ceid%3DUS:en&followup=https://news.google.com
https://news.google.com
https://news.google

In [ ]:
#Task : Visualize the services : - how many services that google have? 
import matplotlib
matplotlib.use('TkAgg')  # Change the backend to 'TkAgg'
import matplotlib.pyplot as plt

# Plot the occurrences of Google services
unique_services = list(set(google_services))
service_counts = [google_services.count(service) for service in unique_services]

plt.figure(figsize=(10, 6))
plt.bar(unique_services, service_counts, color='skyblue')
plt.xlabel('Google Services')
plt.ylabel('Count')
plt.title('Occurrences of Google Services')
plt.xticks(rotation=45)
plt.show()